<a href="https://colab.research.google.com/github/Fatih120/GuildedChatExporter/blob/main/guildedchatexporter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Basic setup
API_KEY = "" # @param {type:"string"}
#SERVER_ID = "" # @param {type:"string"}
CHANNEL_ID = "" # @param {type:"string"}
SAVE_DIRECTORY = "/content/drive/MyDrive/guildedchatexporter/" # @param {type:"string"}
import requests
import json
import os
import re

headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}
SERVER_URL = f'https://www.guilded.gg/api/v1/servers/{SERVER_ID}'
CHANNEL_URL = f'https://www.guilded.gg/api/v1/channels/{CHANNEL_ID}/messages'


In [ ]:
# @title Use Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Get Messages
def get_channel_messages(channel_id, after=None, limit=100):
    params = {
        'limit': limit,
        'after': after
    }
    url = f'https://www.guilded.gg/api/v1/channels/{channel_id}/messages'
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def fetch_all_messages(channel_id):
    all_messages = []
    after = "2015-01-01T00:00:00.000Z"

    while True:
        data = get_channel_messages(channel_id, after=after, limit=100)
        messages = data.get('messages', [])
        all_messages.extend(messages)

        if len(messages) < 100:
            break

        after = messages[-1]['createdAt']

    return all_messages

all_messages = fetch_all_messages(CHANNEL_ID)

#uncomment if you wanna see the output for yourself
#print(json.dumps(all_messages, indent=4))


In [ ]:
# @title Save to HTML File to /guildedchatexporter/(channelid).html

with open(f'{SAVE_DIRECTORY}{CHANNEL_ID}.txt', 'w', encoding='utf-8') as file:
    for message in all_messages:
        created_by = message['createdBy']
        created_at = message['createdAt']
        content = message['content']
        formatted_message = f"[{created_by}] ({created_at}): {content}\n"
        file.write(formatted_message)

print("Chat log saved")

Chat log saved


In [ ]:
# @title Download File Attachments to /guildedchatexporter/(channelid)/attachments/.*
def download_attachments(messages, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    url_pattern = r'!\[(.*?)\]\((.*?)\)'

    for message in messages:
        content = message['content']
        matches = re.findall(url_pattern, content)

        for match in matches:
            url = match[1]
            filename = os.path.join(output_dir, os.path.basename(url))

            try:
                response = requests.get(url)
                response.raise_for_status()

                with open(filename, 'wb') as file:
                    file.write(response.content)

                print(f"Downloaded attachment from {url} to {filename}")
            except requests.exceptions.RequestException as e:
                print(f"Error downloading attachment from {url}: {e}")

all_messages = fetch_all_messages(CHANNEL_ID)
output_dir = f'{SAVE_DIRECTORY}{CHANNEL_ID}'
download_attachments(all_messages, output_dir)

# go grab your files if you're all done
you can shift+click the html and the whole channel attachments and download as zip (or run this code in your own python env if you dont want google)